This notebook shows the implementation of a skorch wrapper class containing the InceptionTime model. Code for the model itself was based on https://github.com/okrasolar/pytorch-timeseries

Author: Christos C. Papadopoulos
https://github.com/Christosc96


In [3]:
import torch
import numpy as np
from sail.models.torch.inceptiontime import InceptionTimeClassifier

Generate data for training the model. The data used originates from the FordA dataset which is included in the InceptionTime paper as well


In [2]:
def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1], ))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)
y_train=y_train.unsqueeze(1)

x_train=x_train.to(torch.float32)
y_train=y_train.to(torch.float32)

To train the model, since we are using a Scorch wrapper, simply call fit after creating it.


In [4]:
model = InceptionTimeClassifier(out_channels=10, bottleneck_channels=10, batch_size=500, max_epochs=20, learning_rate=0.5)
model.fit(x_train, y_train)

  epoch    train_loss     dur
-------  ------------  ------
      1        0.6950  1.8599
      2        0.6930  1.8715
      3        0.6930  2.0367
      4        0.6930  1.9081
      5        0.6930  1.8844
      6        0.6930  1.8808
      7        0.6930  1.9772
      8        0.6929  1.8854
      9        0.6929  1.8985
     10        0.6929  1.8524
     11        0.6929  1.9059
     12        0.6929  1.9283
     13        0.6929  1.8589
     14        0.6929  1.9724
     15        0.6929  1.9674
     16        0.6929  1.8932
     17        0.6928  1.8750
     18        0.6928  1.9269
     19        0.6928  1.8796
     20        0.6928  2.0022


<class 'sail.models.torch.inceptiontime.InceptionTimeClassifier'>[initialized](
  module_=_InceptionModel(
    (blocks): Sequential(
      (0): _InceptionBlock(
        (bottleneck): Conv1dSamePadding(1, 2, kernel_size=(1,), stride=(1,), bias=False)
        (conv_layers): Sequential(
          (0): Conv1dSamePadding(2, 2, kernel_size=(41,), stride=(1,), bias=False)
          (1): Conv1dSamePadding(2, 2, kernel_size=(20,), stride=(1,), bias=False)
          (2): Conv1dSamePadding(2, 2, kernel_size=(10,), stride=(1,), bias=False)
        )
        (batchnorm): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (residual): Sequential(
          (0): Conv1dSamePadding(1, 2, kernel_size=(1,), stride=(1,), bias=False)
          (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (1): _InceptionBlock(
        (bottleneck): Conv1dSamePadding(2, 2, kernel_size